In [ ]:
import numpy as np
import random

In [ ]:
class MyEnsemble(c, X, y):
    def __init__(self):
        self.c = c
        self._w = []
        self.X = X
        self.y = y
        self._p = []
        self._bags
    def predict(self){
        totals = self._p.T * self.w
        return(np.amax(totals))    
    }
    def fit(self){
        i = 0
        for n in self.c:
            for classifier, bags in zip(self.c[i], self._bags[i]):
                # want to fit each classifier with each vector in bag
                classifier.predict_proba(bags)
            i+=1
    }
    def _predict_c(self){
        self._p = np.hstack(predict_proba(self.X))
    }
    def _sample(self){
        self._bags = self.X[np.random.randint(len(self.c), 20), :]
    }